In [62]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_train = pd.read_csv(Path("adults_data","adults_cleaned_train_data.csv"))
df_test = pd.read_csv(Path("adults_data","adults_cleaned_test_data.csv"))

In [5]:
def data_transformation_for_KNN(data_to_trans):

    data = data_to_trans.copy()
    data['income'] = data['income'].replace({"<=50K":0,
                                            ">50K":1})
    
    pd.set_option('future.no_silent_downcasting', True)
    data = data.infer_objects(copy=False)
    
    to_compare = data.copy()
    occupation_indices= data[data['occupation'].isnull()].index
    region_indices = data[data['region'].isnull()].index    
    
    encoder_dict = {}

    for column in data.columns:
        if data[column].dtype in ["object","category"]:
            encoder = LabelEncoder()
            encoder_dict[column] = encoder
            data[column] = encoder.fit_transform(data[column])
    miss_occ_as_nr = data.loc[occupation_indices[0],"occupation"]
    miss_reg_as_nr = data.loc[region_indices[0],"region"]
    data['occupation'] = data['occupation'].replace({miss_occ_as_nr:None})
    data['region'] = data['region'].replace({miss_reg_as_nr:None})
    
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1]
    
    return X, Y, encoder_dict, to_compare

In [6]:
def KNN_imputation(data_to_impute):
    
    data = data_to_impute.copy()
    
    KNN_X, KNN_Y, encoders, to_compare = data_transformation_for_KNN(data)
    
    imputer = KNNImputer(n_neighbors=5, weights="uniform", metric="nan_euclidean")
                                
    imputer.fit(KNN_X)
    X_transformed = pd.DataFrame(imputer.transform(KNN_X),
                                 columns=KNN_X.columns)
    
    for column in encoders.keys():
        X_transformed[column] = encoders[column].inverse_transform(X_transformed[column].astype(int))
    
    X_transformed.index = KNN_X.index
    return X_transformed, to_compare
    

### train imputation

In [7]:
dencoded, org = KNN_imputation(df_train) 

dencoded['income'] = org['income']
dencoded.to_csv(Path("adults_data","adults_imputed_train_data.csv"), index=None)

C:\Users\wypyc\AppData\Local\Temp\ipykernel_11992\3450783541.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['income'] = data['income'].replace({"<=50K":0,


### test imputation

In [8]:
total_df = pd.concat([dencoded, df_test], ignore_index=True)
dencoded_test, org_test = KNN_imputation(total_df) 

dencoded_test['income'] = org_test['income']
dencoded_test.to_csv(Path("adults_data","adults_imputed_test_data.csv"), index=None)

In [9]:
(dencoded_test != org_test).sum()

age                 0
workclass           0
fnlwgt              0
education-num       0
marital-status      0
occupation          1
relationship        0
race                0
sex                 0
capital             0
hours-per-week      0
region            192
income              0
dtype: int64

In [10]:
df_test.isnull().sum()

age                 0
workclass           0
fnlwgt              0
education-num       0
marital-status      0
occupation          1
relationship        0
race                0
sex                 0
capital             0
hours-per-week      0
region            192
income              0
dtype: int64

# dummies

In [115]:
def dummies(data_to_trans):

    data = data_to_trans.copy()
    data['income'] = data['income'].replace({"<=50K":0,
                                            ">50K":1})
    
    pd.set_option('future.no_silent_downcasting', True)
    data = data.infer_objects(copy=False)
    
    to_compare = data.copy()
    occupation_indices= data[data['occupation'].isnull()].index
    region_indices = data[data['region'].isnull()].index    
    
    encoder_dict = {}

    for column in data.columns:
        if data[column].dtype in ["object","category"]:
            data_encoded = pd.get_dummies(data[column], prefix=column, dummy_na=True).astype(int)
            data = pd.concat([data, data_encoded], axis=1)
            data.pop(column)
            """
    miss_occ_as_nr = data.loc[occupation_indices[0],"occupation"]
    miss_reg_as_nr = data.loc[region_indices[0],"region"]
    data['occupation'] = data['occupation'].replace({miss_occ_as_nr:None})
    data['region'] = data['region'].replace({miss_reg_as_nr:None})"""
    data.drop(columns=['sex_Male', 'fnlwgt'], inplace=True)
   
    Y = data.loc[:,'income']
    X = data.drop(columns="income")
    
    return X, Y, encoder_dict, to_compare

In [116]:
x, y, dict, comp = dummies(df_train)

In [117]:
x.columns

Index(['education-num', 'capital', 'hours-per-week', 'age_0-17', 'age_18-24',
       'age_25-30', 'age_31-40', 'age_41-50', 'age_51-60', 'age_61-70',
       'age_71+', 'age_nan', 'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'workclass_nan', 'marital-status_Divorced',
       'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'marital-status_nan', 'occupation_Adm-clerical',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupatio

In [106]:
df_train.isnull().sum()

age                 0
workclass           0
fnlwgt              0
education-num       0
marital-status      0
occupation          9
relationship        0
race                0
sex                 0
capital             0
hours-per-week      0
region            770
income              0
dtype: int64

In [113]:
pd.get_dummies(df_train['region'], dummy_na=True, prefix='region')['region_nan'].sum()

770